In [1]:
"""

""" 

'\n\n'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import math
import random

MIN_LIST_LEN = 8
MAX_LIST_LEN = 8
MAX_STEPS = 32

CONTEXT_LEN=128
EMBED_DIM=128
NUM_LAYERS=3

SUCCESS_REWARD = 5.0
#FAILURE_REWARD = -0.3
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
CORRECT_POS_REWARD = 5.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.9999
SHORTTERM_GAMMA = 0.7

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 6000
LR_SCHEDULER_GAMMA = 0.98
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
BASE_DIR = '/home/mcwave/code/autocode/datasets/rl_merge_transformer_easy/'
OUTPUT_FOLDER = f'list{MIN_LIST_LEN}-{MAX_LIST_LEN}_transformer{NUM_LAYERS}_{EMBED_DIM}_gamma07_step{MAX_STEPS}_merge_2_sorted'
OUTPUT_DIR = BASE_DIR + OUTPUT_FOLDER

# Define the updated vocabulary
vocab = {
    'Comparison': 0,
    'Take1': 1,
    'Take2': 2,
    'less': 3,
    'equal': 4,
    'more': 5,
    '0': 6,
    '1': 7,
    '2': 8,
    '3': 9,
    '4': 10,
    '5': 11,
    '6': 12,
    '7': 13,
    '8': 14,
    '9': 15,
    '10': 16,
    '11': 17,
    '12': 18,
    '13': 19,
    '14': 20,
    '15': 21,
    'len1': 22,
    'len2': 23,
    'len3': 24,
    'len4': 25,
    'len5': 26,
    'len6': 27,
    'len7': 28,
    'len8': 29,
    'len9': 30,
    'len10': 31,
    'len11': 32,
    'len12': 33,
    'len13': 34,
    'len14': 35,
    'len15': 36,
    'len16': 37,
}
inv_vocab = {v: k for k, v in vocab.items()}
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def generate_unique_random_list(length, min_value=1, max_value=100):
    """
    Generates a list of unique random integers within a specified range.
    If duplicates are detected, the list is regenerated.
    
    Args:
        length (int): The length of the list to generate.
        min_value (int): The minimum possible value (inclusive).
        max_value (int): The maximum possible value (inclusive).
        
    Returns:
        list: A list of unique random integers.
    """
    if length > (max_value - min_value + 1):
        raise ValueError("Length exceeds the range of unique values available.")
    
    while True:
        random_list = [random.randint(min_value, max_value) for _ in range(length)]
        if len(set(random_list)) == length:
            return random_list

# def merge_sort(arr):
#     """
#     Perform a traditional merge sort on `arr`.
#     Returns a tuple (sorted_array, compares, copies) where:
#     - sorted_array is the fully sorted version of arr.
#     - compares is the total number of element-to-element comparisons performed.
#     - copies is the total number of element copy operations performed (to the merged output).
#     """
#     n = len(arr)
#     if n <= 1:
#         # For a single-element array, no compares, one could argue 0 copies here since 
#         # we do no merging, but we'll say no copies occur during sorting a single element.
#         return arr[:], 0, 0  # Make a copy of arr so we return a consistent value
    
#     mid = n // 2
#     left_half, left_compares, left_copies = merge_sort(arr[:mid])
#     right_half, right_compares, right_copies = merge_sort(arr[mid:])
    
#     merged, merge_compares, merge_copies = merge(left_half, right_half)
    
#     # Accumulate totals
#     total_compares = left_compares + right_compares + merge_compares
#     total_copies = left_copies + right_copies + merge_copies
    
#     return merged, total_compares, total_copies

def merge(left, right):
    """
    Merge two sorted lists `left` and `right` into a single sorted list.
    Returns (merged_list, compares, copies).
    """
    i = 0
    j = 0
    merged = []
    compares = 0
    copies = 0
    
    # Merge while both halves have elements
    while i < len(left) and j < len(right):
        # Compare elements from left and right
        compares += 1
        if left[i] <= right[j]:
            # Copy element from left
            merged.append(left[i])
            copies += 1
            i += 1
        else:
            # Copy element from right
            merged.append(right[j])
            copies += 1
            j += 1
    
    # Copy any remaining elements from left
    while i < len(left):
        merged.append(left[i])
        copies += 1
        i += 1
    
    # Copy any remaining elements from right
    while j < len(right):
        merged.append(right[j])
        copies += 1
        j += 1
    
    return compares + copies

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN // 2, MAX_LIST_LEN // 2) * 2
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.list = generate_unique_random_list(self.length)
        list1, list2 = sorted(self.list[0:self.length//2]), sorted(self.list[self.length//2:])
        self.list = list1 + list2
        self.original_list = list(self.list)
        self.optimal_steps = merge(list(list1), list(list2))
        self.sorted_list = sorted(self.list)
        self.output_idx = 0
        self.correct_cnt = 0
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_original_list(self):
        return self.original_list
    
    def get_list_len(self):
        return len(self.list)
    
    def get_optimal_steps(self):
        return self.optimal_steps
    
    def get_correct_cnt(self):
        return self.correct_cnt

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Take1'] or action == vocab['Take2']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            if action == vocab['Take1']:
                final_pos = self.sorted_list.index(self.list[index1])
            else:
                final_pos = self.sorted_list.index(self.list[index2])
            dest_pos = self.output_idx
            self.output_idx += 1
            is_correct_pos = False
            if final_pos == dest_pos:
                is_correct_pos = True
                self.correct_cnt += 1                
            if self.correct_cnt == len(self.list) - 1:
                reward = SUCCESS_REWARD
                self.done = True
            elif is_correct_pos:
                reward = CORRECT_POS_REWARD
            else:
                reward = STEP_REWARD
                self.done = True
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps or self.output_idx > len(self.list):
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [3]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=CONTEXT_LEN):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=EMBED_DIM, nhead=8, num_layers=NUM_LAYERS):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
def compute_bellman_returns(raw_rewards, gamma):
    bellman_returns = []
    R = 0
    for r in raw_rewards[::-1]:
        R = r + gamma * R
        bellman_returns.insert(0, R)
    return bellman_returns

# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=LR_SCHEDULER_GAMMA)
    
    # Optionally, load a checkpoint
    #checkpoint_path = os.path.join(OUTPUT_DIR, "ckpt_120000_0.6380_14.00_vs_14.29.pth")
    #load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    success_step_cnts = []
    optimal_step_cnts = []
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        comparisons = []
        steps = 0
        
        state = 'expect_action'
        done = False
        success = False

        while not done: #and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Take1'], vocab['Take2']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 6

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action in [vocab['Take1'],vocab['Take2']]:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [action]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    steps += 1
                    if done and reward == SUCCESS_REWARD:
                        success = True
                    if done and episode % 100 == 0:
                        print("Original:", env.get_original_list())
                        print(decode(input_tokens, inv_vocab))
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), 
                                    int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list = env.step(action_tokens)
                steps += 1
                if done and reward == SUCCESS_REWARD:
                    success = True
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        # Post processing of each episode
        success_rewards = [0.0] * len(rewards)
        if success: 
            num_successes += 1
            success_rewards[-1] = SUCCESS_REWARD
            success_step_cnts.append(steps)
            optimal_step_cnts.append(env.get_optimal_steps())
        #else:
        #    success_rewards[-1] = FAILURE_REWARD
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns1 = compute_bellman_returns(rewards, SHORTTERM_GAMMA)
            returns2 = compute_bellman_returns(success_rewards, LONGTERM_GAMMA)
            returns = torch.tensor(np.array(returns1) + np.array(returns2)).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, len{env.get_list_len()}, loss:{loss.item():.4f}, correct:{env.get_correct_cnt()}, {'succeed' if success else 'fail'}, steps:{steps}, opt steps:{env.get_optimal_steps()}, total reward:{sum(rewards):.4f}, {t2-t1:.4f} sec")
            
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            #assert len(success_step_cnts) == num_successes
            success_rate = num_successes / episode_cnt
            avg_steps = sum(success_step_cnts) / (len(success_step_cnts) + 0.001)
            avg_optimal_steps = sum(optimal_step_cnts) / (len(optimal_step_cnts) + 0.001)
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}_vs_{avg_optimal_steps:.2f}.pth")
            #
            # Reduce the lr
            scheduler.step()
            # Optionally, log the learning rate
            current_lr = scheduler.get_last_lr()[0]
            print(f"Learning rate = {current_lr:.6f}")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Checkpoint loaded from /home/mcwave/code/autocode/datasets/rl_merge_transformer_easy/list8-8_transformer3_128_gamma07_step32_merge_2_sorted/ckpt_120000_0.6380_14.00_vs_14.29.pth, resuming from episode 120000
Original: [10, 38, 54, 83, 14, 20, 45, 58]
len8 Comparison 0 4 less Take1 Comparison 1 4 more Take2 Comparison 1 5 more Take2 Comparison 1 6 less Take1 Comparison 2 6 more Take2 Comparison 2 7 less Take1 Comparison 2 3 less Take2
Episode 0, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.3812 sec
Episode 1, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0581 sec
Episode 2, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0551 sec
Episode 3, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0549 sec
Episode 4, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0548 sec
Episode 5, len8, loss:0.0000, 

Episode 73, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0546 sec
Episode 74, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0545 sec
Episode 75, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0546 sec
Episode 76, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0545 sec
Episode 77, len8, loss:0.0000, correct:2, fail, steps:6, opt steps:15, total reward:8.8000, 0.0247 sec
Episode 78, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0548 sec
Episode 79, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0544 sec
Episode 80, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0544 sec
Episode 81, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0543 sec
Episode 82, len8, loss:0.0000, corre

Episode 150, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0549 sec
Episode 151, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0546 sec
Episode 152, len8, loss:0.0000, correct:3, fail, steps:8, opt steps:15, total reward:13.5000, 0.0320 sec
Episode 153, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0540 sec
Episode 154, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0543 sec
Episode 155, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0548 sec
Episode 156, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0546 sec
Episode 157, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0544 sec
Episode 158, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0547 sec
Episode 159, len8, loss:0.0000, cor

Episode 229, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0549 sec
Episode 230, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0546 sec
Episode 231, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0543 sec
Episode 232, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0545 sec
Episode 233, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0553 sec
Episode 234, len8, loss:0.0000, correct:3, fail, steps:8, opt steps:14, total reward:13.5000, 0.0322 sec
Episode 235, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0540 sec
Episode 236, len8, loss:0.0000, correct:2, fail, steps:6, opt steps:15, total reward:8.8000, 0.0246 sec
Episode 237, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0537 sec
Episode 238, len8, loss:0.0000, co

...

Episode 5350, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0551 sec
Episode 5351, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0550 sec
Episode 5352, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0548 sec
Episode 5353, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0546 sec
Episode 5354, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0551 sec
Episode 5355, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0550 sec
Episode 5356, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0547 sec
Episode 5357, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0548 sec
Episode 5358, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0553 sec
Episode 5359, len8, lo

Episode 5426, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0551 sec
Episode 5427, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0547 sec
Episode 5428, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0549 sec
Episode 5429, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:13, total reward:32.9000, 0.0547 sec
Episode 5430, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0551 sec
Episode 5431, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0549 sec
Episode 5432, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0549 sec
Episode 5433, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0549 sec
Episode 5434, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0550 sec
Episode 5435, len8, lo

Episode 5501, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0551 sec
Episode 5502, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0549 sec
Episode 5503, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0547 sec
Episode 5504, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0546 sec
Episode 5505, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0550 sec
Episode 5506, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0549 sec
Episode 5507, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0546 sec
Episode 5508, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0547 sec
Episode 5509, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0551 sec
Episode 5510, len8, lo

Episode 5581, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:13, total reward:32.9000, 0.0540 sec
Episode 5582, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0543 sec
Episode 5583, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0544 sec
Episode 5584, len8, loss:0.0000, correct:2, fail, steps:6, opt steps:14, total reward:8.8000, 0.0246 sec
Episode 5585, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0537 sec
Episode 5586, len8, loss:0.0000, correct:2, fail, steps:6, opt steps:15, total reward:8.8000, 0.0249 sec
Episode 5587, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0537 sec
Episode 5588, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0541 sec
Episode 5589, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:14, total reward:32.9000, 0.0544 sec
Episode 5590, len8, loss:0.00

Episode 5657, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0547 sec
Episode 5658, len8, loss:0.0000, correct:2, fail, steps:6, opt steps:12, total reward:8.8000, 0.0249 sec
Episode 5659, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0537 sec
Episode 5660, len8, loss:0.0000, correct:3, fail, steps:8, opt steps:15, total reward:13.5000, 0.0319 sec
Episode 5661, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0539 sec
Episode 5662, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0544 sec
Episode 5663, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0546 sec
Episode 5664, len8, loss:0.0000, correct:7, succeed, steps:14, opt steps:15, total reward:32.9000, 0.0543 sec
Episode 5665, len8, loss:0.0000, correct:6, fail, steps:14, opt steps:15, total reward:27.6000, 0.0545 sec
Episode 5666, len8, loss: